In [1]:
# Install a pip package in the current Jupyter kernel
import sys


### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
#from scipy.optimize import curve_fit
from tqdm import tqdm
import pandas as pd

### Query
from astroquery.sdss import SDSS

In [2]:
df = pd.read_csv("data/sdssdr14-wd-catalog/Table2.csv")

ps = []
ms = []
fs = []

for i in range(len(df['P-M-F'])):
    tpmf = df['P-M-F'][i].split('-')
        
    ps.append(int(tpmf[0]))
    ms.append(int(tpmf[1]))
    fs.append(int(tpmf[2]))
        
df['plate'] = ps
df['mjd'] = ms
df['fiber'] = fs

C:\Users\arsen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print(type(df['SNg'][2]))

<class 'numpy.int64'>


In [22]:
from astropy.coordinates import SkyCoord
from astropy import units as u

i = df['SNg'].tolist().index(max(df['SNg']))

dothing = False

if dothing:
    xids = []
    
    for i in range(len(df['SNg'])):
        xid = SDSS.get_spectra_async(plate=df['plate'][i], fiberID=df['fiber'][i], mjd=df['mjd'][i])
        xids.append(xid)
else:
    xid = SDSS.get_spectra_async(plate=df['plate'][i], fiberID=df['fiber'][i], mjd=df['mjd'][i])
    
url = str(xid[0]).split(' ')
print(url)
print((str(xids[0]).split(' ')[4]).split('-'))

from numpy import asarray
from numpy import save

if dothing:
    xids2 = np.array(xids)
    xids2.tofile('data/wd-spectra.txt', sep = ',')

['Downloaded', 'object', 'from', 'URL', 'https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/2682/spec-2682-54401-0422.fits', 'with', 'ID', '2267986761952']
['https://data.sdss.org/sas/dr14/sdss/spectro/redux/26/spectra/0266/spec', '0266', '51602', '0314.fits']


In [38]:
wds = {}
urls = []
objids = []
p = []
m = []
f = []

xids = xids2

if xids is not None:
    for i in range(len(xids)):
        #print(str(xids[i][0]).split(' ')[4])
        try:        
            urls.append(str(xids[i][0]).split(' ')[4])
            objids.append(str(xids[i][0]).split(' ')[7])
            p.append( (str(xids[i][0]).split(' ')[4]).split('-')[1] )
            m.append( (str(xids[i][0]).split(' ')[4]).split('-')[2] )
            f.append( ((str(xids[i][0]).split(' ')[4]).split('-')[3]).split('.')[0] )
        except:
            pass
    
    wds['objectID'] = objids
    wds['plate'] = p
    wds['mjd'] = m
    wds['fiberID'] = f
    wds['fits'] = urls
    
    white_dwarf = pd.DataFrame.from_dict(wds)
    white_dwarf.to_csv('data/white_dwarves.dat')
else:
    try:
        white_dwarf = pd.read_csv('data/white_dwarves/dat')
    except:
        pass

37406


In [37]:
from astropy.io import fits
star = fits.open(url)

star[0].header['SPEC_ID']

'   3019779622465202176'

In [41]:
#!{sys.executable} -m pip install sdss
import sdss

sql = """SELECT TOP 5
specObjID, bestObjID, mjd, plate, fiberID, z
FROM SpecObj"""

df2 = sdss.sql2df(sql)
df2

,specObjID,bestObjID,mjd,plate,fiberID,z
0,3154868093696632832,1237663268796432489,54326,2802,351,-0.0001514952
1,3154870567597795328,1237663268796432699,54326,2802,360,-6.223064E-05
2,3154870017841981440,1237663268796497989,54326,2802,358,-8.696781E-05
3,3154868918330353664,1237663268796498001,54326,2802,354,-0.0003906398
4,3154866719307098112,1237663268796498065,54326,2802,346,-0.000104708


In [ ]:
##### https://www.sdss.org/dr17/spectro/spectro_basics/
# https://data.sdss.org/datamodel/files/SPECTRO_REDUX/RUN2D/PLATE4/spCFrame.html

header = star[1].header
data = star[1].data

wavelength = 10**data['loglam']
flux = data['flux']
ivar = data['ivar']

plt.figure(figsize=(20,8))

plt.plot(wavelength, flux)
plt.grid()
plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
plt.xlabel(r'Wavelength [A]')
plt.title('White Dwarf Light Curve')
ax = plt.gca()

plt.show()

In [ ]:
from lmfit.models import GaussianModel, ConstantModel

ha = np.where(np.logical_and([wavelength > 6500], [wavelength < 6630]))[1]

x = wavelength[ha]
y = flux[ha]

model = (ConstantModel(prefix='offset_') +
         GaussianModel(prefix='p_'))

params = model.make_params(offset_c=350, p_amplitude=-300., p_sigma=100, p_center=6550) 
init = model.eval(params, x=x)
result = model.fit(y, params, x=x)

print(result.fit_report())

plt.figure(figsize=(15,6))

plt.plot(wavelength[ha], flux[ha], label='Spectrum Data')
plt.plot(wavelength[ha], result.best_fit, c='black', label='Gaussian Fit')
plt.xlim((6500,6630))
#plt.ylim((150,400))
plt.grid()
plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
plt.xlabel(r'Wavelength [A]')
plt.title(r'White Dwarf $H\alpha$ Absorbtion Line')
ax = plt.gca()

plt.legend()
plt.show()

In [ ]:
c = 299792458 #m/s
h_alpha = 6564.61

lambda_obs = result.params['p_center'].value
dlambda_obs = result.params['p_center'].stderr

v_app = c * ((lambda_obs - h_alpha) / h_alpha) * 1e-3
dv_app = np.abs(v_app * 1e3) * np.sqrt((dlambda_obs / lambda_obs)**2 )

print('Observed speed by redshift: %.2f +/- %.2f km/s' % (v_app, dv_app))

In [ ]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
#!{sys.executable} -m pip install tensorflow


import sys
sys.path.append('../wdtools') # Your path should contain the parent GitHub repository

In [ ]:
import wdtools

gfp = wdtools.GFP(resolution = 2, specclass = 'DA')

labels, e_labels, redchi = gfp.fit_spectrum(wavelength, flux, ivar,
                                mcmc = True, nwalkers = 25, burn = 50, ndraws = 50,
                                plot_init = True, make_plot = True, plot_corner = True, savename = 'example',
                                verbose = True,
                                lines = ['beta', 'gamma', 'delta', 'eps', 'h8'])

In [ ]:
sp = wdtools.SpecTools()

rv, e_rv = sp.get_line_rv(wavelength, flux, ivar, centroid = h_alpha,
                          distance = 100, edge = 15,
                          nmodel = 2, plot = True)

In [ ]:
print('Observed speed by redshift (WDTools): %.2f +/- %.2f km/s' % (rv, e_rv))
print('Observed speed by redshift (Gaussian): %.2f +/- %.2f km/s' % (v_app, dv_app))

In [ ]:
print('Teff = %i ± %i Kelvin' % (labels[0], e_labels[0]))
print('logg = %.2f ± %.2f log[cm/s^2]' % (labels[1], e_labels[1]))
print('RV = %.1f ± %.1f km/s' % (labels[2], e_labels[2]))
print('reduced chi^2 = %.2f' % (redchi))